# Семинар 8
## Линейная регрессия

In [ ]:
import statsmodels.api as sm
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

plt.style.use('ggplot')

In [ ]:
%matplotlib inline

## Постановка

По 1260 опрошенным имеются следующие данные:

* заработная плата за час работы, $;
* опыт работы, лет;
* образование, лет;
* внешняя привлекательность, в баллах от 1 до 5;
* бинарные признаки: пол, семейное положение, состояние здоровья (хорошее/плохое), членство в профсоюзе, цвет кожи (белый/чёрный), занятость в сфере обслуживания (да/нет).

Требуется оценить влияние внешней привлекательности на уровень заработка с учётом всех остальных факторов.

Hamermesh D.S., Biddle J.E. (1994) Beauty and the Labor Market, American Economic Review, 84, 1174–1194.

Данные:

In [ ]:
raw = pd.read_csv("beauty.csv", sep=";", index_col=False) 
raw.head()

Посмотрим на матрицу диаграмм рассеяния по количественным признакам:

In [ ]:
pd.tools.plotting.scatter_matrix(raw[['wage', 'exper', 'educ', 'looks']], alpha=0.2, 
                                 figsize=(15, 15), diagonal='hist')
pylab.show()

Оценим сбалансированность выборки по категориальным признакам:

In [ ]:
print raw.union.value_counts()
print raw.goodhlth.value_counts()
print raw.black.value_counts()
print raw.female.value_counts()
print raw.married.value_counts()
print raw.service.value_counts()

У каждого признака все значения встречаются достаточно много раз, так что всё в порядке.

## Предобработка

In [ ]:
data = raw

Посмотрим на распределение целевого признака — уровня заработной платы: 

In [ ]:
##

Посмотрим на распределение оценок привлекательности: 

В группах looks=1 и looks=5 слишком мало наблюдений. Превратим признак looks в категориальный и закодируем с помощью фиктивных переменных:

Данные теперь:

## Построение модели

### Простейшая модель

Построим линейную модель по всем признакам.

In [ ]:
m1 = smf.ols('wage ~ exper + union + goodhlth + black + female + married +'\
                    'service + educ + belowavg + aboveavg', 
             data=data)
fitted = m1.fit()
print fitted.summary()

Посмотрим на распределение остатков:

In [ ]:
plt.figure(figsize(16,7))
plt.subplot(121)
sc.stats.probplot(fitted.resid, dist="norm", plot=pylab)
plt.subplot(122)
np.log(fitted.resid).plot.hist()
plt.xlabel('Residuals', fontsize=14)
pylab.show()

Оно скошенное, как и исходный признак. В таких ситуациях часто помогает перейти от регрессии исходного признака к регрессии его логарифма.

### Логарифмируем отклик

Видим, что у остатков, да и у признака заработная плата - тежелый хвост. В таких случаях обычно помогает логарифт!!

Теперь стало лучше. Посмотрим теперь на зависимость остатков от непрерывных признаков (образование и опыт):

### Добавляем квадрат опыта работы

Добавим квадратичную зависимость по опыту работы!

И посмотрим, как изменилась зависимость остатоков от непрерывных признаков

Используем критерий Бройша-Пагана для проверки гомоскедастичности ошибок:

In [ ]:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

Ошибки гетероскедастичны, значит, значимость признаков может определяться неверно. Сделаем поправку Уайта:

`fitted = m4.fit(cov_type='HC1')`

### Удаляем незначимые признаки

В предыдущей модели незначимы: цвет кожи, здоровье, семейное положение. Удалим их. Индикатор привлекательности выше среднего тоже незначим, но удалять его не будем, потому что это одна из переменных, по которым на нужно в конце ответить на вопрос.

Посмотрим, не стала ли модель от удаления трёх признаков значимо хуже, с помощью критерия Фишера:

In [ ]:
print "F=%f, p=%f, k1=%f" % m4.fit().compare_f_test(m5.fit())
# m4 - Большая модель
# m5 - Вложенная в нее модель

Не стала.

Проверим, нет ли наблюдений, которые слишком сильно влияют на регрессионное уравнение:

In [ ]:
plt.figure(figsize(8,7))
plot_leverage_resid2(fitted)
pylab.show()

In [ ]:
data.loc[[1122]]

In [ ]:
data.loc[[269]]

## Выводы

Итоговая модель объясняет 40% вариации логарифма отклика. 

In [ ]:
plt.figure(figsize(16,7))
plt.subplot(121)
scatter(data['wage'],np.exp(fitted.fittedvalues))
plt.xlabel('Wage', fontsize=14)
plt.ylabel('Exponentiated predictions', fontsize=14)
plt.xlim([0,50])

plt.subplot(122)
scatter(np.log(data['wage']),fitted.fittedvalues)
plt.xlabel('Log wage', fontsize=14)
plt.ylabel('Predictions', fontsize=14)
plt.xlim([0,4])
pylab.show()

При интересующих нас факторах привлекательности стоят коэффициенты -0.1307 (ниже среднего) и -0.0010 (выше среднего). 

Поскольку регрессия делалась на логарифм отклика, интерпретировать их можно как прирост в процентах. С учётом дополнительных факторов представители генеральной совокупности, из которой взята выборка, получают в среднем:

* на 13% меньше, если их привлекательность ниже среднего (p=0.001, 95% доверительный интервал — [5,21]%);
* столько же, если их привлекательность выше среднего (p=0.972, 95% доверительный интервал — [-6,6]%).

# Другие методы работы с выборосами

Загрузим данные о различных характеристиках автомобиля. Будем пытаться предсказывать его стоимость по пробегу:

In [ ]:
df_train = pd.read_csv('http://bit.ly/1gIQs6C')

n = df_train.shape[0]

In [ ]:
X_train = df_train.mileage.values.reshape(-1, 1)
y_train = df_train.price.values

Добавим выбросы

In [ ]:
X_train = np.r_[X_train, [[250000+np.random.rand()*10000]]]
y_train = np.r_[y_train, 16000+np.random.randn()*1000]

Обучим 2 модели: одна на "чистых" данных, другая на всех данных (вместе с выбросами).

Нарисуем предсказания моделей

## RANSAC регрессия

Идея метода RANdom SAmple Consensus (RANSAC) заключается в многократном обучении модели на случайном наборе точек из исходных данных с последующим выбором лучшей модели.

То есть:
* Задаем функцию потерь
* Задаем порог $\theta$ для остатков при котором наблюдения начинают относится к выбросам
* Задаем правило останова

Шаги алгоритма следующие
1. Взять случайные K точек и обучить на них модель M
2. Сравнить ошибки на остальных точких с порогом $\theta$ и отнести к выбросам или внутренним точкам
3. Обучить модель на всех внутренних точках, оценить качество на внутренних точках
4. Повторить 1-3 пока не наступит правило останова. 
5. Вывод: модель с лучшим качеством

In [ ]:
from sklearn.linear_model import RANSACRegressor

## Robust Estimators

Идея робастных методов заключается во взвешивании остатков модели таким образом, чтобы большие значения вносили меньший вклад в оценку параметров.

Таким образом, вместо минимизации квадрата остатков $$ L(\beta_0,\beta_1,\dots) = \frac{1}{2n}\sum^{n}_{i=1}(\hat{y}^{(i)} - y^{(i)})^2$$
Будут минимизироваться взвешенные остатки $$ L_w(\beta_0,\beta_1,\dots) = \frac{1}{2n}\sum^{n}_{i=1}\rho(\hat{y}^{(i)} - y^{(i)}),$$
где $\rho(\cdot)$ - некоторая взвешивающая функция.

In [ ]:
c = 4.685
support = np.linspace(-3*c, 3*c, 1000)
tukey = sm.robust.norms.TukeyBiweight(c=c)
plt.plot(support, tukey(support))
plt.ylim(.1, -4.1)

In [ ]:
model_robust = sm.RLM(y_train, sm.add_constant(X_train),
                      M=sm.robust.norms.TukeyBiweight())
model_robust = model_robust.fit()